In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import IPython as IP
import IPython.display as ipd
from keras.models import Sequential, load_model

In [2]:
!unzip '/content/drive/MyDrive/Gen AI/SoundClassification/EmergencyvsNonEmergencyVehicleSoundClassification-200414-193707/test.zip'

Archive:  /content/drive/MyDrive/Gen AI/SoundClassification/EmergencyvsNonEmergencyVehicleSoundClassification-200414-193707/test.zip
   creating: Test/
  inflating: test.csv                
  inflating: Test/1002.wav           
  inflating: Test/1005.wav           
  inflating: Test/1009.wav           
  inflating: Test/1010.wav           
  inflating: Test/1011.wav           
  inflating: Test/1012.wav           
  inflating: Test/1013.wav           
  inflating: Test/1016.wav           
  inflating: Test/1019.wav           
  inflating: Test/102.wav            
  inflating: Test/1020.wav           
  inflating: Test/1023.wav           
  inflating: Test/1031.wav           
  inflating: Test/1033.wav           
  inflating: Test/1035.wav           
  inflating: Test/1037.wav           
  inflating: Test/1039.wav           
  inflating: Test/1041.wav           
  inflating: Test/1046.wav           
  inflating: Test/1050.wav           
  inflating: Test/1053.wav           
  inflating:

In [3]:
sample_sub_df = pd.read_csv('/content/drive/MyDrive/Gen AI/SoundClassification/EmergencyvsNonEmergencyVehicleSoundClassification-200414-193707/sample_submission_EsSVCId.csv')
sample_sub_df.head()

,Class,ID
0,gun_shot,5
1,gun_shot,7
2,gun_shot,8
3,gun_shot,9
4,gun_shot,13


In [4]:
sample_sub_df.shape

(3297, 2)

In [6]:
# Iterate through all audio files and extract MFCC
features = []
frames_max = 0
counter = 0
total_samples = len(sample_sub_df)
n_mfcc = 40

for index, row in sample_sub_df.iterrows():
    id = str(row["ID"])
    file_name = '/content/Test/' + str(id) + '.wav'

    # Extract MFCCs (do not add padding)
    # mfccs = helpers.get_mfcc(file_path, 0, n_mfcc)
    y, sr = librosa.load(file_name, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

    # Save current frame count
    num_frames = mfccs.shape[1]

    # Add row (feature / label)
    features.append(mfccs)

    # Update frames maximum
    if (num_frames > frames_max):
        frames_max = num_frames

    # Notify update every N files
    if (counter == 500):
        print("Status: {}/{}".format(index+1, total_samples))
        counter = 0

    counter += 1

print("Finished: {}/{}".format(index, total_samples))

/usr/local/lib/python3.10/dist-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


Status: 501/3297
Status: 1001/3297
Status: 1501/3297
Status: 2001/3297
Status: 2501/3297
Status: 3001/3297
Finished: 3296/3297


In [7]:
features[0].shape

(40, 609)

In [8]:
frames_max

1501

In [14]:
a = []
for i in features:
  a.append(i.shape[1])

In [16]:
a = pd.DataFrame({'len': a})
a.head()

,len
0,609
1,345
2,192
3,345
4,345


In [18]:
a = a.sort_values(by=['len'])
a.head()

,len
2783,9
2115,15
1273,17
2104,17
461,19


In [21]:
a[a['len']==751].shape

(203, 1)

In [22]:
def pad_features(feature_list, frames_max):
    # Determine the number of features (x)
    num_features = feature_list[0].shape[0]

    # Initialize a list to hold the padded features
    padded_list = []

    for features in feature_list:
        num_frames = features.shape[1]
        if num_frames < frames_max:
            # Create a zero matrix with shape (num_features, frames_max)
            padded_features = np.zeros((num_features, frames_max))
            # Copy the original features into the zero matrix
            padded_features[:, :num_frames] = features
        else:
            # If the original features have more frames than frames_max, truncate them
            padded_features = features[:, :frames_max]

        # Append the padded (or truncated) features to the list
        padded_list.append(padded_features)

    return padded_list

In [23]:
frames_max = 751
padded_features = pad_features(features, frames_max)

In [24]:
padded_features[0].shape

(40, 751)

In [25]:
# Verify shapes
print("Raw features length: {}".format(len(features)))
print("Padded features length: {}".format(len(padded_features)))

Raw features length: 3297
Padded features length: 3297


In [26]:
# Convert features (X) and labels (y) to Numpy arrays
X = np.array(padded_features)

In [27]:
# Optionally save the features to disk
np.save("/content/drive/MyDrive/Gen AI/SoundClassification/EmergencyvsNonEmergencyVehicleSoundClassification-200414-193707/X-mfcc-test_ass", X)

In [2]:
X_test = np.load("/content/drive/MyDrive/Gen AI/SoundClassification/EmergencyvsNonEmergencyVehicleSoundClassification-200414-193707/X-mfcc-test_ass.npy")

In [8]:
print(X_test.shape)

(3297, 40, 751)


In [9]:
# How data should be structured
num_rows = 40
num_columns = 751
num_channels = 1

# Reshape to fit the network input (channel last)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

In [10]:
print(X_test.shape)

(3297, 40, 751, 1)


In [11]:
model_file = 'simple-train-nb3.keras'
model_path = '/content/drive/MyDrive/Gen AI/SoundClassification/EmergencyvsNonEmergencyVehicleSoundClassification-200414-193707/'+model_file

In [12]:
model = load_model(model_path)

In [13]:
y_test_pred = model.predict(X_test)

104/104 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


In [14]:
y_test_pred

array([[1.8226942e-08, 1.6839836e-09, 4.8851083e-09, ..., 9.9996662e-01,
        9.5464144e-11, 1.2729743e-09],
       [2.2765605e-03, 1.7804629e-04, 2.3250461e-01, ..., 9.0239868e-05,
        2.9828848e-02, 6.3993111e-02],
       [2.8463102e-05, 1.6410789e-03, 1.0384551e-05, ..., 9.0000358e-05,
        8.6354821e-06, 1.0681256e-04],
       ...,
       [1.0767534e-02, 2.8833767e-02, 4.6794387e-04, ..., 4.5274472e-01,
        3.4238302e-04, 6.6312477e-03],
       [4.3150084e-04, 2.6792435e-03, 6.0576312e-02, ..., 8.0580416e-05,
        5.9176511e-01, 4.5636237e-02],
       [1.6170844e-02, 2.0227565e-04, 2.6735142e-02, ..., 7.6789642e-03,
        6.1479920e-01, 4.9513173e-03]], dtype=float32)

In [15]:
y_test_pred = np.argmax(y_test_pred, axis=1)

In [16]:
y_test_pred

array([7, 3, 4, ..., 4, 8, 8])

In [18]:
sample_sub_df = pd.read_csv('/content/drive/MyDrive/Gen AI/SoundClassification/EmergencyvsNonEmergencyVehicleSoundClassification-200414-193707/sample_submission_EsSVCId.csv')
sample_sub_df.head()

,Class,ID
0,gun_shot,5
1,gun_shot,7
2,gun_shot,8
3,gun_shot,9
4,gun_shot,13


In [19]:
sample_sub_df['y_pred'] = y_test_pred

In [20]:
sample_sub_df.head()

,Class,ID,y_pred
0,gun_shot,5,7
1,gun_shot,7,3
2,gun_shot,8,4
3,gun_shot,9,2
4,gun_shot,13,9


In [21]:
labels_ = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark',
       'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren',
       'street_music']
nums_ = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [22]:
d = dict()
for i in range(len(labels_)):
  d[nums_[i]] = labels_[i]

In [23]:
d

{0: 'air_conditioner',
 1: 'car_horn',
 2: 'children_playing',
 3: 'dog_bark',
 4: 'drilling',
 5: 'engine_idling',
 6: 'gun_shot',
 7: 'jackhammer',
 8: 'siren',
 9: 'street_music'}

In [24]:
sample_sub_df['Class'] = sample_sub_df['y_pred'].apply(lambda x: d[x])

In [25]:
sample_sub_df.head()

,Class,ID,y_pred
0,jackhammer,5,7
1,dog_bark,7,3
2,drilling,8,4
3,children_playing,9,2
4,street_music,13,9


In [26]:
sample_sub_df = sample_sub_df[['Class', 'ID']]
sample_sub_df.head()

,Class,ID
0,jackhammer,5
1,dog_bark,7
2,drilling,8
3,children_playing,9
4,street_music,13


In [27]:
sample_sub_df.to_csv('/content/drive/MyDrive/Gen AI/SoundClassification/EmergencyvsNonEmergencyVehicleSoundClassification-200414-193707/UrbanSoundSubmission.csv', index=False)